# Junior Data Scientist Take-Home Task: Product Catalogue Creation


# Imports

In [ ]:
import pandas as pd
import numpy as np
import re


## 1. Data Ingestation

In [ ]:
# load the CSV files
bd_tech = pd.read_csv('data/bd_technologies.csv')
ts_tech = pd.read_csv('data/ts_technologies.csv')

# reset the index of both DataFrames
bd_tech.reset_index(drop=True, inplace=True)
ts_tech.reset_index(drop=True, inplace=True)

## 2.1. Data Exploration

In [ ]:
# display the first few rows of each DataFrame
display(bd_tech.head()), display(ts_tech.head())

In [ ]:
# display the column names of each DataFrame
display(bd_tech.columns, ts_tech.columns)

In [ ]:
# summary statistics for each DataFrame
display(bd_tech.describe(), ts_tech.describe())


In [ ]:
# display the general information about each DataFrame
bd_tech.info(verbose=True), ts_tech.info()


In [ ]:
# Function to generate data quality report
def data_quality_report(df):
    return pd.DataFrame({
        "Missing Values": df.isnull().sum(),
        "Percentage Missing": (df.isnull().sum() / len(df)) * 100,
        "Data Type": df.dtypes
    })
# Generate data quality reports
bd_quality_report = data_quality_report(bd_tech)
ts_quality_report = data_quality_report(ts_tech)

display(bd_quality_report), display(ts_quality_report)

**Initial Data Quality Issues:**

> bd_technologies.csv

- Major missing values in the headquarters column (~43.8% missing).

- Minimal missing values in seller_website and categories.
 * Also its important to note that these values are not accurate until the inconsistencies with data types and missing values are addressed. 
 
> ts_technologies.csv

- Significant missing values in url (~76.8%) and description (~73.9%).




In [ ]:
# display duplicates in each DataFrame
def display_duplicates(df):
    duplicates = df[df.duplicated()]
    if not duplicates.empty:
        print(f"Duplicates found:\n{duplicates}")
    else:
        print("No duplicates found.")
display_duplicates(bd_tech)
display_duplicates(ts_tech)

## 2.2. Data Cleaning

## 2.2.1. Missing Values


In [ ]:
# replace field that's entirely space (or empty) with NaN
ts_tech = ts_tech.replace(r'^\s*$', np.nan, regex=True)
bd_tech = bd_tech.replace(r'^\s*$', np.nan, regex=True)

### 2.2.2. Data Types

In [ ]:
# convert all column data types to string for consistency in db_tech
def convert_to_string(df):
    for col in df.columns:
        df[col] = df[col].astype('string')
    return df
bd_tech = convert_to_string(bd_tech)

In [ ]:
bd_tech.info()

In [ ]:
# convert the columns 1 to 9 in ts_tech to string
def convert_ts_tech_to_string(df):
    for col in df.columns[1:9]:
        df[col] = df[col].astype('string')
    return df
ts_tech = convert_ts_tech_to_string(ts_tech)
ts_tech.info()

### 2.2.3. Standardize the URL 

While the primary identifiers are obviously product name and description, we can also consider URLs and Seller Websites which are often unique and can strongly confirm product matches. Therefore it is essential we have standardized URLs (removal of protocols, www and trailing slashes).

In [ ]:
# standardize the url and seller_website columns by removing protocols, wwww and trailing slashes
def standardize_urls(df, url_col):
    df[url_col] = df[url_col].str.lower()  # convert to lowercase
    df[url_col] = df[url_col].str.replace(r'^https?://', '', regex=True)  # remove http/https
    df[url_col] = df[url_col].str.replace(r'^www\.', '', regex=True)  # remove www.
    df[url_col] = df[url_col].str.rstrip('/')  # remove trailing slashes
    return df
bd_tech = standardize_urls(bd_tech, 'seller_website')
ts_tech = standardize_urls(ts_tech, 'url')

In [ ]:
# pick 20 random entries from the seller_website column for quality control
bd_tech['seller_website'].sample(5)


In [ ]:
ts_tech['url'].sample(5)

### 2.2.4. Clean the categories column

In [ ]:
# clean the categories column in bd_tech - removing brackets and quotes
bd_tech['categories'] = (
    bd_tech['categories']
    .str.replace(r'[\[\]"]', '', regex=True)           # Remove brackets and quotes
    .str.replace(r',\s*', ', ', regex=True)            # Ensure single space after comma
)


In [ ]:
bd_tech.categories.sample(5)

### 2.2.5. Clean the software_product_id and parent_category_slug column

`software_product_id` column in `bd_tech` and `parent_category_slug` column in `ts_tech` have string values seperated by hyphens replace them with space instead. 

In [ ]:
def remove_hyphens(column):
    # Replace hyphens with spaces in the specified column
    return column.str.replace(r'(?<=[A-Za-z])-(?=[A-Za-z])', ' ', regex=True)

# Apply the function to the 'software_product_id' column in bd_tech 
bd_tech['software_product_id'] = remove_hyphens(bd_tech['software_product_id'])

# Apply the function to the 'slug', 'category_slug' and 'parent_category_slug' column in ts_tech
ts_tech['slug'] = remove_hyphens(ts_tech['slug'])
ts_tech['category_slug'] = remove_hyphens(ts_tech['category_slug'])
ts_tech['parent_category_slug'] = remove_hyphens(ts_tech['parent_category_slug'])


In [ ]:
# Sample the cleaned 'software_product_id' column in bd_tech
bd_tech['software_product_id'].sample(5)

In [ ]:
# randomly sample 5 entries from the 'slug', 'category_slug', 'parent_category_slug' columns in ts_tech
ts_tech[['slug', 'category_slug', 'parent_category_slug']].sample(5)

In [ ]:
# combine 'category' and 'parent_category' in ts_tech into a single column called 'categories' and drop the original two columns
ts_tech_clean = ts_tech.copy()
ts_tech_clean['categories_ts'] = ts_tech_clean['category'] + ', ' + ts_tech_clean['parent_category']
ts_tech_clean.drop(columns=['category', 'parent_category', 'parent_category_slug', 'category_slug', 'slug'], inplace=True) 
# remove leading and trailing spaces from the 'categories' column in ts_tech
ts_tech_clean['categories_ts'] = ts_tech_clean['categories_ts'].str.strip()

# combine main_category and categories in bd_tech into a single column called 'categories' and drop the original two columns
bd_tech_clean = bd_tech.copy()
bd_tech_clean['categories_bd'] = bd_tech_clean['main_category'] + ', ' + bd_tech_clean['categories']
bd_tech_clean.drop(columns=['main_category', 'categories', 'software_product_id'], inplace=True)
# remove leading and trailing spaces from the 'categories' column in bd_tech
bd_tech_clean['categories_bd'] = bd_tech_clean['categories_bd'].str.strip()
# display the first few rows of each DataFrame after cleaning
display(bd_tech_clean.head()), display(ts_tech_clean.head())

In [ ]:
ts_tech_clean.info()

In [ ]:
bd_tech_clean.info()

## 3. Product Deduplicaton

### 3.1. Data Engineering

Here we will standardize our text. This ensures that data across records follows the same format, which is crucial for reliable comparison. urls have already been standardized. Next we clean text by converting to lowercase, removing punctutations, extra whitespaces and stopwords, and lastly applying lemmatization. 

In [ ]:
# Import necessary libraries
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('wordnet')

bd_tech_processed = bd_tech_clean.copy()
ts_tech_processed = ts_tech_clean.copy()




In [ ]:
# Define cleaning function
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def clean_text(text):
    if pd.isna(text):
        return ""
    text = text.lower()
    text = re.sub(r'[^\w\s]', ' ', text)
    words = [lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words]
    return ' '.join(words)

# Define preprocessing function for combined columns
def preprocess(df, columns):
    return df[columns].fillna('').agg(' '.join, axis=1).apply(clean_text)

# Apply preprocessing to datasets
bd_tech_processed['combined'] = preprocess(bd_tech_processed, ['product_name', 'description', 'seller_website', 'categories_bd'])
ts_tech_processed['combined'] = preprocess(ts_tech_processed, ['name', 'description', 'url', 'categories_ts'])

# Display the first few results
bd_tech_processed[['product_name', 'combined']].head(), ts_tech_processed[['name', 'combined']].head()

In [ ]:
# save the cleaned DataFrames to CSV files
bd_tech_processed.to_csv('data/bd_technologies_cleaned.csv', index=False)
ts_tech_processed.to_csv('data/ts_technologies_cleaned.csv', index=False)

### 3.2. Fuzzy Matching (Rapid Initial Matching)

Fuzzy Matching is a quick tool for calculating similarities between strings to help identify values that are "close enough". This method allows variations and/or inconsistencies in data (i.e., typos, different spelling) to be considered similar. We calculate the Levenshtein distance between our strings with a threshold of 0.85. If the similarity score exceeds the threshold, the data records are considered a fuzzy match and can be linked. 

In [ ]:
# load the cleaned DataFrames
bd_tech_processed = pd.read_csv('data/bd_technologies_cleaned.csv')
ts_tech_processed = pd.read_csv('data/ts_technologies_cleaned.csv')

In [ ]:
# from fuzzywuzzy import fuzz, process

# def fuzzy_match(ts_text, bd_choices, threshold=85):
#     match, score = process.extractOne(ts_text, bd_choices)
#     return (match, score) if score >= threshold else (None, score)

# ts_tech_processed['fuzzy_match'] = ts_tech_processed['combined'].apply(lambda x: fuzzy_match(x, bd_tech_processed['combined'].tolist()))

# fuzzywuzzy was taking too long to run, so we will switch to thefuzz instead 

# from thefuzz import fuzz, process
# from tqdm import tqdm

# # enable tqdm for pandas
# tqdm.pandas()

# def fuzzy_match_fast(ts_text, bd_choices, threshold=85):
#     match, score = process.extractOne(ts_text, bd_choices, scorer=fuzz.token_set_ratio)
#     return (match, score) if score >= threshold else (None, score)

# # Apply to TS dataset with progress bar
# ts_tech_processed['fuzzy_match'] = ts_tech_processed['combined'].progress_apply(
#     lambda x: fuzzy_match_fast(x, bd_tech_processed['combined'].tolist())
# )
# The above code was still taking too long, so we will try using RapidFuzz for faster fuzzy matching

# Use RapidFuzz for faster fuzzy matching
# from tqdm import tqdm
# from rapidfuzz import fuzz, process

# # enable tqdm for pandas
# tqdm.pandas()

# def fuzzy_match_fast(ts_text, bd_choices, threshold=85):
#     match = process.extractOne(ts_text, bd_choices, scorer=fuzz.token_set_ratio)
#     return match if match else (None, 0)

# # Apply to TS dataset with progress bar
# ts_tech_processed['fuzzy_match'] = ts_tech_processed['combined'].progress_apply(
#     lambda x: fuzzy_match_fast(x, bd_tech_processed['combined'].tolist())
# )

# Even this would take 4 hrs

Both Fuzzy Matching methods took longer than 8 hrs to complete. We will try to speed up the fuzzy matching process by applying below methods: 
- Limit to top 10 cadidates with TF-IDF Pre-Filtering by cosine similarity
- Apply fuzzy matching only on those 
- use RapidFuzz instead of thefuzz and fuzzywuzzy


This method will provide a semantic matching beyond simple string similarity. 

TF-IDF (Term Frequency-Inverse Document Frequency) and cosine similarity are used together to measure the similarity between text documents. TF-IDF converts text into numerical vectors, while cosine similarity calculates the similarity between those vectors. A higher cosine similarity value indicates greater similarity between documents. 
1. TF-IDF:
Term Frequency (TF): Measures how often a term appears in a document. 
Inverse Document Frequency (IDF): Measures how rare a term is across a corpus of documents. 
TF-IDF Value: The product of TF and IDF, weighting terms based on their importance in a specific document and the entire corpus. 
Vector Representation: TF-IDF converts each document into a numerical vector, where each dimension represents a term and the value in that dimension is the term's TF-IDF score. 
2. Cosine Similarity: 
Concept:
Measures the similarity between two vectors by calculating the cosine of the angle between them. 
Calculation:
Takes the dot product of the two vectors and divides it by the product of their magnitudes. 
Interpretation:
A cosine similarity value of 1 indicates identical vectors (perfectly similar), 0 indicates orthogonal vectors (no similarity), and -1 indicates completely opposite vectors (perfectly dissimilar). 
Usage:
Used to compare the numerical vectors created by TF-IDF, determining how closely two documents match based on their term frequencies and importance. 
In summary:
TF-IDF creates numerical representations of documents, and cosine similarity compares these representations to determine how similar the documents are based on their content. 

In [ ]:
# TF-IDF Vectorization
from sklearn.feature_extraction.text import TfidfVectorizer
def compute_tfidf(df, column):
    vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(df[column])
    return tfidf_matrix, vectorizer

# Compute TF-IDF for both datasets
bd_tfidf, bd_vectorizer = compute_tfidf(bd_tech_processed, 'combined')
ts_tfidf, ts_vectorizer = compute_tfidf(ts_tech_processed, 'combined')

# Cosine Similarity
from sklearn.metrics.pairwise import cosine_similarity
cos_sim_matrix = cosine_similarity(ts_tfidf, bd_tfidf)

In [ ]:
from tqdm import tqdm
from rapidfuzz import fuzz, process

# Limit to top 5 candidates and apply fuzzy matching
tqdm.pandas()
fuzzy_matches = []

for i in tqdm(range(len(ts_tech_processed)), desc="Matching top candidates"):
    ts_text = ts_tech_processed.iloc[i]['combined']
    top_indices = cos_sim_matrix[i].argsort()[-5:][::-1]
    candidates = bd_tech_processed.iloc[top_indices]['combined'].tolist()
    result = process.extractOne(ts_text, candidates, scorer=fuzz.token_set_ratio)
    fuzzy_matches.append(result)

# Store results
ts_tech_processed['fuzzy_match'] = fuzzy_matches

# Display sample results
ts_tech_processed[['name', 'fuzzy_match']].head()

In [ ]:
# Extract matched product names from fuzzy match results
# Assuming each entry in 'fuzzy_match' is a tuple: (matched_text, score)
matched_names = ts_tech_processed['fuzzy_match'].dropna().apply(lambda x: x[0])
match_scores = ts_tech_processed['fuzzy_match'].dropna().apply(lambda x: x[1])

# Create a DataFrame for matched pairs
matched_df = ts_tech_clean.loc[matched_names.index, ['name', 'description', 'url', 'categories_ts']].copy()
matched_df['matched_to'] = matched_names.values
matched_df['match_score'] = match_scores.values

# Merge with bd_tech on 'product_name' (which matched_to points to)
bd_matched = bd_tech_clean[['product_name', 'description', 'seller_website', 'categories_bd']].copy()
bd_matched.columns = ['matched_to', 'bd_description', 'bd_url', 'bd_category']

# Join TS matches with BD products
merged = pd.merge(matched_df, bd_matched, on='matched_to', how='left')
display(merged.head())

In [ ]:
# Consolidate matched fields (favor longer description, combine URLs)
merged['product_name'] = merged['name']
merged['description'] = merged.apply(lambda row: row['description'] if len(str(row['description'])) > len(str(row['bd_description'])) else row['bd_description'], axis=1)
merged['url'] = merged.apply(lambda row: row['url'] if pd.notna(row['url']) and row['url'] != '' else row['bd_url'], axis=1)
merged['category'] = merged.apply(lambda row: row['category'] if pd.notna(row['category']) and row['category'] != '' else row['bd_category'], axis=1)
merged['source'] = 'matched'

# Select relevant columns
master_matched = merged[['product_name', 'description', 'url', 'category', 'source', 'match_score']]

# Unmatched TS entries
matched_ts_names = set(matched_df['name'])
ts_unmatched = ts_tech[~ts_tech['name'].isin(matched_ts_names)]
ts_only = ts_unmatched[['name', 'description', 'url', 'category']].copy()
ts_only.columns = ['product_name', 'description', 'url', 'category']
ts_only['source'] = 'ts_only'
ts_only['match_score'] = None

# Unmatched BD entries
matched_bd_names = set(matched_df['matched_to'])
bd_unmatched = bd_tech[~bd_tech['product_name'].isin(matched_bd_names)]
bd_only = bd_unmatched[['product_name', 'description', 'seller_website', 'main_category']].copy()
bd_only.columns = ['product_name', 'description', 'url', 'category']
bd_only['source'] = 'bd_only'
bd_only['match_score'] = None

# Combine all parts into master catalogue
master_catalogue = pd.concat([master_matched, ts_only, bd_only], ignore_index=True)

display(master_catalogue.head())

In [ ]:
# Display the info on source=matched data only 
matched_stats = master_catalogue[master_catalogue['source'] == 'matched'].info()
display(matched_stats)

In [ ]:
# Save the master catalogue to a CSV file
master_catalogue.to_csv('data/master_catalogue_fuzzy_matching.csv', index=False)

### 3.3. Hyprid feature-based entity resolution with fuzzy token-set ratio

In [ ]:
import pandas as pd
from recordlinkage import Index
from rapidfuzz import fuzz
from tqdm.auto import tqdm


In [ ]:
# loaad the cleaned ts and bd tech DataFrames
bd = pd.read_csv('data/bd_technologies_cleaned.csv')
ts = pd.read_csv('data/ts_technologies_cleaned.csv')

In [ ]:
bd.info(), ts.info()

In [ ]:
# Block on categories to reduce candidate pairs
indexer = Index()
indexer.block(left_on='categories_bd', right_on='categories_ts')
candidates = indexer.index(bd, ts)

In [ ]:
# For each candidate pair, compute fuzzy score on `combined`
matches = []
scores = []

In [ ]:
for bd_idx, ts_idx in tqdm(candidates, desc="Blocking + Fuzzy matching"):
    bd_text = bd.at[bd_idx, 'combined']
    ts_text = ts.at[ts_idx, 'combined']
    score = fuzz.token_set_ratio(bd_text, ts_text)
    if score >= 85:
        matches.append((bd_idx, ts_idx))
        scores.append(score)

In [ ]:
# Build a DataFrame of matched index‐pairs + score
matches_df = pd.DataFrame(matches, columns=['bd_idx','ts_idx'])
matches_df['match_score'] = scores

In [ ]:
# Extract matched rows, reset index to align with matches_df
bd_matched = bd.loc[matches_df['bd_idx']].reset_index(drop=True)
ts_matched = ts.loc[matches_df['ts_idx']].reset_index(drop=True)
bd_matched.index = matches_df.index
ts_matched.index = matches_df.index

In [ ]:
# Fuse matched rows into one DataFrame (suffixing columns)
matched_full = pd.concat([
    bd_matched.add_suffix('_bd'),
    ts_matched.add_suffix('_ts')
], axis=1)
matched_full['match_score'] = matches_df['match_score']
matched_full['source'] = 'matched'

In [ ]:
# Prepare TS-only records
ts_unmatched_idx = ts.index.difference(matches_df['ts_idx'])
ts_unmatched = ts.loc[ts_unmatched_idx].reset_index(drop=True)
ts_only = ts_unmatched.add_suffix('_ts')
ts_only['match_score'] = pd.NA
ts_only['source'] = 'ts_only'
# add blank BD columns
for col in bd.columns:
    ts_only[col + '_bd'] = pd.NA
    
# Prepare BD-only records
bd_unmatched_idx = bd.index.difference(matches_df['bd_idx'])
bd_unmatched = bd.loc[bd_unmatched_idx].reset_index(drop=True)
bd_only = bd_unmatched.add_suffix('_bd')
bd_only['match_score'] = pd.NA
bd_only['source'] = 'bd_only'
# add blank TS columns
for col in ts.columns:
    bd_only[col + '_ts'] = pd.NA


In [ ]:
# Reorder TS-only and BD-only to match matched_full columns
all_cols = list(matched_full.columns)
ts_only = ts_only[all_cols]
bd_only = bd_only[all_cols]

In [ ]:
# Concatenate into final master catalogue
master_catalogue_hybrid = pd.concat(
    [matched_full, ts_only, bd_only],
    ignore_index=True
)

# eorder columns: put product identifiers front
cols = (
    ['product_name_bd','name_ts']
    + [c for c in all_cols if c not in ('product_name_bd','name_ts')]
)
master_catalogue_hybrid = master_catalogue_hybrid[cols]

In [ ]:
master_catalogue_hybrid.to_csv('master_catalogue_hybrid.csv', index=False)
display("Master catalogue shape:", master_catalogue_hybrid.shape)
display(master_catalogue_hybrid.head())

In [ ]:
# display master catalogue with matched entries and the info
master_catalogue_hybrid[master_catalogue_hybrid['source'] == 'matched'].info()